# RAG

## Requirements

In [18]:
%%capture
!pip install transformers accelerate bitsandbytes langchain langchain-community sentence-transformers faiss-gpu pandas gdown

## Dataset

In [19]:
!gdown --fuzzy https://drive.google.com/file/d/1Lq2zVJlN_B4kUAu4VafQ4jXMIQiAR9vI/view?usp=sharing

Downloading...
From (original): https://drive.google.com/uc?id=1Lq2zVJlN_B4kUAu4VafQ4jXMIQiAR9vI
From (redirected): https://drive.google.com/uc?id=1Lq2zVJlN_B4kUAu4VafQ4jXMIQiAR9vI&confirm=t&uuid=3a67c0cf-5e02-485b-a6c0-9b92bb41d894
To: /kaggle/working/IMDB_crawled.json
100%|█████████████████████████████████████████| 292M/292M [00:01<00:00, 174MB/s]


## Config

In [20]:
class Config:
    EMBEDDING_MODEL_NAME="thenlper/gte-base"
    LLM_MODEL_NAME="HuggingFaceH4/zephyr-7b-beta"
    K = 5 # top K retrieval

## Preprocessing

In [21]:
import pandas as pd

df = pd.read_json('IMDB_crawled.json')

In [22]:
import os

os.makedirs('data', exist_ok=True)

# preprocess your data and only store the needed data as the context window for embedding model is limited
df = df[['id', 'title', 'first_page_summary', 'genres']]

df.to_csv('data/imdb.csv', index=False)

## Vectorizer

load the CSV file and vectorize the rows using HuggingFaceEmbeddings.
Store the results using FAISS vectorstore.
Save the vectorestore in a pickle file for future usages.

In [23]:
import pickle
import numpy as np
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.vectorstores.utils import DistanceStrategy
from langchain.vectorstores.faiss import FAISS

from langchain_community.embeddings import HuggingFaceEmbeddings

# load the csv
file_path = 'data/imdb.csv'
csv_loader = CSVLoader(file_path)
documents = csv_loader.load()

# load the embeddings model
embeddings = HuggingFaceEmbeddings(model_name=Config.EMBEDDING_MODEL_NAME, 
encode_kwargs={"normalize_embeddings": True})

# save embed the documents using the model in a vectorstore
vectorstore = FAISS.from_documents(documents, embeddings)

with open("data/vectorstore.pkl", "wb") as f:
  pickle.dump(vectorstore, f)

print("Vectorstore saved to data/vectorstore.pkl")

/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Vectorstore saved to data/vectorstore.pkl


load the vectorstore as a retriever.

In [24]:
# with open("data/vectorstore.pkl", "rb") as f:
#     vectorstore = pickle.load(f)

# load the retriever from the vectorstore
retriever = vectorstore.as_retriever(top_k=5)

## LLM

load the quantized LLM.

In [25]:
import torch

from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from transformers import pipeline

from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline

# load the quantization config
bnb_config = BitsAndBytesConfig()

model = AutoModelForCausalLM.from_pretrained(Config.LLM_MODEL_NAME, quantization_config=bnb_config, device_map="cuda:0")
tokenizer = AutoTokenizer.from_pretrained(Config.LLM_MODEL_NAME)

# init the pipeline
READER_LLM = pipeline("text-generation", model=model, tokenizer=tokenizer, max_length=40000)

llm = HuggingFacePipeline(
    pipeline=READER_LLM,
)

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

initialize the prompt template for the query chain. query chain is used to get a query from the chat history. you may change the prompt as you like to get better results.

In [26]:
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.chains import LLMChain

class LoggerStrOutputParser(StrOutputParser):
    def parse(self, text: str) -> str:
        # process the LLM output
        print(f"QUERY: {text}")
        return text

query_transform_prompt = PromptTemplate(
    input_variables=["messages"],
    template="""<|system|>You are a helpful assistant.
{messages}
<|user|>
give me the search query about the above conversation.
<|assistant|>"""
)

# init the query chain
query_transforming_retriever_chain = LLMChain(
    prompt=query_transform_prompt,
    output_parser=LoggerStrOutputParser(),
    llm =llm
)

initialize the main retrieval chain that gives the resulting documents to LLM and gets the output back.

In [27]:
from langchain.chains.combine_documents import create_stuff_documents_chain

from langchain_core.runnables import RunnablePassthrough

prompt = PromptTemplate(
    input_variables=["context", "messages"],
    template="""<|system|>You are a helpful assistant.

Here are the movies you MUST choose from:

{context}
-----------------
{messages}
<|assistant|>""")

# helped from https://python.langchain.com/v0.1/docs/modules/data_connection/retrievers/
# init the retriver chain
context_chain = (
    {"context": retriever, "messages": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

write the conversation helper class for easier testing.

In [28]:
class Conversation:
    def __init__(self):
        self.messages = []

    def add_assistant_message(self, message):
        self.messages.append(('assistant', message))

    def add_user_message(self, message):
        self.messages.append(('user', message))

    def get_messages(self):
        # concatenate the messages with the roles in the instruction format
        return "\n".join([f"{role}: {msg}" for role, msg in self.messages])

    def chat(self, message):
        self.add_user_message(message)
        messages = self.get_messages()
        # invoke the chain
        response = context_chain.invoke(messages)
        self.add_assistant_message(response)
        return response

## Test

talk with the RAG to see how good it performs.

In [29]:
c = Conversation()
A = c.chat('give me a cool gangster movie')
print(A)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


<|system|>You are a helpful assistant.

Here are the movies you MUST choose from:

[Document(page_content="id: tt0765429\ntitle: American Gangster\nfirst_page_summary: An outcast New York City cop is charged with bringing down Harlem drug lord Frank Lucas, whose real life inspired this partly biographical film.\ngenres: ['Biography', 'Crime', 'Drama']", metadata={'source': 'data/imdb.csv', 'row': 2433}), Document(page_content='id: tt0102603\ntitle: Oscar\nfirst_page_summary: A gangster attempts to keep the promise he made to his dying father: that he would give up his life of crime and "go straight".\ngenres: [\'Comedy\', \'Crime\']', metadata={'source': 'data/imdb.csv', 'row': 9856}), Document(page_content='id: tt1166168\ntitle: Al imbrator\nfirst_page_summary: A remake of "scarface",tells the turbulant world of drug dealing,damaged personal relationships,and the rise and the fall of a drug lord.\ngenres: [\'Crime\', \'Drama\']', metadata={'source': 'data/imdb.csv', 'row': 4197}), Doc

In [30]:
A = c.chat('give me a newer one')
print(A)

<|system|>You are a helpful assistant.

Here are the movies you MUST choose from:

[Document(page_content="id: tt0765429\ntitle: American Gangster\nfirst_page_summary: An outcast New York City cop is charged with bringing down Harlem drug lord Frank Lucas, whose real life inspired this partly biographical film.\ngenres: ['Biography', 'Crime', 'Drama']", metadata={'source': 'data/imdb.csv', 'row': 2433}), Document(page_content="id: tt0026393\ntitle: 'G' Men\nfirst_page_summary: James Cagney helped jump-start the gangster genre as The Public Enemy. Outcries against movies that glorified underworld criminals put Cagney on the side of the law in 'G' Men.\ngenres: ['Crime', 'Drama', 'Film-Noir']", metadata={'source': 'data/imdb.csv', 'row': 4203}), Document(page_content='id: tt1166168\ntitle: Al imbrator\nfirst_page_summary: A remake of "scarface",tells the turbulant world of drug dealing,damaged personal relationships,and the rise and the fall of a drug lord.\ngenres: [\'Crime\', \'Drama\'